# Setup

## Library import

In [1]:
# Data manipulation
# ==============================================================================
import pandas as pd
import numpy as np

# Visualization
# ==============================================================================
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

# Stattiscals tests
# ==============================================================================
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold

# from statsmodels.graphics.tsaplots import plot_pacf

# GreyBox [own package]
# ==============================================================================
from greyboxmodel.model import DarkGreyModel, Ti, TiTe, TiTh, TiTeTh, TiTeThRia
from greyboxmodel.fit import darkgreyfit

# Warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Autoreload extension
# ==============================================================================
%reload_ext autoreload
%autoreload 2

In [2]:
# Local functions
# ==============================================================================
from sklearn.metrics import mean_squared_error


def rmse(*args, **kwargs):
    return mean_squared_error(*args, **kwargs) ** 0.5


# Data import
We retrieve all the required data for the analysis.

In [3]:
# Duration of a record
# ==============================================================================
rec_duration = 1 # hour

# Loading temperature Data
# ==============================================================================
input_df = pd.read_csv('../data/demo_data.csv', index_col=0, parse_dates=True)
input_df.head()

,Ph,Ti,Ta,Th
2019-12-23 00:00:00+00:00,0.000000,18.1375,5.4,4.60
2019-12-23 01:00:00+00:00,0.000000,18.1000,5.1,4.45
2019-12-23 02:00:00+00:00,0.000000,18.0125,5.0,4.15
2019-12-23 03:00:00+00:00,97.223561,17.9625,4.8,4.10
2019-12-23 04:00:00+00:00,76.202250,17.9875,5.5,68.15


Read some demo data.  
* Ph: Heating system power output
* Ta: Ambient temperature
* Ti: Internal temperature

We are also setting a set of custom fields in our input data: `Ti0`, `Th0` and `Te0`. These fields define the initial conditions for `Ti`, `Th` and `Te` for each record of the input data respectively. E.g. if a sub-model is trained based on the 10-20 records of the training data, the initial conditions for the above params will be set by the 10. record of the input data. 

In [4]:
# Setting the initial conditions
# ==============================================================================
input_df['Ti0'] = input_df['Ti']
input_df['Th0'] = input_df['Ti']
input_df['Te0'] = input_df['Ti'] - 2

input_X = input_df[['Ph', 'Ta', 'Ti0', 'Te0', 'Th0']]
input_y = input_df['Ti']

print(f'Input X shape: {input_X.shape}, input y shape: {input_y.shape}')

Input X shape: (792, 5), input y shape: (792,)


In [5]:
# Splitting the data into train and test
# ==============================================================================
X_train, X_test, y_train, y_test = train_test_split(input_X, input_y, test_size=5 / 33, shuffle=False)

print(f'Train: X shape: {X_train.shape}, y shape: {y_train.shape}')
print(f'Test: X shape: {X_test.shape}, y shape: {y_test.shape}')

Train: X shape: (672, 5), y shape: (672,)
Test: X shape: (120, 5), y shape: (120,)


Set up the model training parameters for each model instance that we want to spin up in the pipeline.  

The `Ti0` param is the initial condition for the internal temperature at t=0 - this is set to the first record of `X_train['Ti0']` as described above and is fixed, hence `vary: False`.  

The `Te0` param is the initial condition for the building envelope temperature at t=0 - - this is set to the first record of `X_train['Te0']` as described above and is NOT fixed, hence `vary: True`.  

The `Th0` param is the initial condition for the heating system temperature at t=0 - - this is set to the first record of `X_train['Th0']` as described above and is NOT fixed, hence `vary: True`.  

`Ci`, `Ce`, `Rie` and `Ria` params are the initial conditions for these thermal parameters. As these will be fit by the model training their default is `vary: True`. The values for these params' initial conditions are set arbitrarily to `1` it is assumed that no estimates have been calculated for them (e.g. based on building physical properties).

Note that we've added the `Ch` and `Rih` params with set initial conditions not allowed to vary. These are the RC params for the heating system and their values have been precalculated from the heating system's time constant to make this example simpler.


In [6]:
train_params_Ti = {
    'Ti0': {'value': X_train.iloc[0]['Ti0'], 'vary': False},
    'Ci': {'value': 1},
    'Ria': {'value': 1},
}

train_params_TiTe = {
    'Ti0': {'value': X_train.iloc[0]['Ti0'], 'vary': False},
    'Te0': {'value': X_train.iloc[0]['Ti0'], 'vary': True},
    'Ci': {'value': 1},
    'Ce': {'value': 1},
    'Rie': {'value': 1},
    'Rea': {'value': 1},
}

train_params_TiTh = {
    'Ti0': {'value': X_train.iloc[0]['Ti0'], 'vary': False},
    'Th0': {'value': X_train.iloc[0]['Th0'], 'vary': False},
    'Ci': {'value': 1},
    'Ch': {'value': 2.55, 'vary': False},
    'Ria': {'value': 1},
    'Rih': {'value': 0.65, 'vary': False}
}

train_params_TiTeTh = {
    'Ti0': {'value': X_train.iloc[0]['Ti0'], 'vary': False},
    'Te0': {'value': X_train.iloc[0]['Te0'], 'vary': True, 'min': 10, 'max': 25},
    'Th0': {'value': X_train.iloc[0]['Th0'], 'vary': False},
    'Ci': {'value': 1},
    'Ch': {'value': 2.55, 'vary': False},
    'Ce': {'value': 1},
    'Rie': {'value': 1},
    'Rea': {'value': 1},
    'Rih': {'value': 0.65, 'vary': False}
}

train_params_TiTeThRia = {
    'Ti0': {'value': X_train.iloc[0]['Ti0'], 'vary': False},
    'Te0': {'value': X_train.iloc[0]['Te0'], 'vary': True, 'min': 10, 'max': 25},
    'Th0': {'value': X_train.iloc[0]['Th0'], 'vary': False},
    'Ci': {'value': 1},
    'Ch': {'value': 2.55, 'vary': False},
    'Ce': {'value': 1},
    'Rie': {'value': 1},
    'Rea': {'value': 1},
    'Ria': {'value': 1},
    'Rih': {'value': 0.65, 'vary': False}
}

Set up our initial conditions parameters map for testing

In [7]:
ic_params_map = {
    'Ti0': lambda X_test, y_test, train_result: y_test.iloc[0],
    'Th0': lambda X_test, y_test, train_result: y_test.iloc[0],
    'Te0': lambda X_test, y_test, train_result: train_result.var['Te'][-1],
}

We can instantiate a range of models to be evaluated - in this case we are setting up 5 different models starting from the simplest `Ti` to `TiTeThRia` with medium complexity.

In [8]:
models = [
    Ti(train_params_Ti, rec_duration),
    TiTe(train_params_TiTe, rec_duration),
    TiTh(train_params_TiTh, rec_duration),
    TiTeTh(train_params_TiTeTh, rec_duration),
    TiTeThRia(train_params_TiTeThRia, rec_duration)
]

We need to split our training data set into managable chunks for the prefit stage. `Sklearn` is quite helpful here again, in case we wanted to define the prefit sets to be 1 day (24 hours) long:

In [9]:
prefit_splits = KFold(n_splits=int(len(X_train) / 24), shuffle=False).split(X_train)

To evaluate the performance of our models we will need to define an error metric. Again, there is a wide range of options available from `sklearn`:

In [10]:
error_metric = rmse

We can define a prefit "survival threshold" to trim the number of model fits during the training process. Any models achieving a higher error during the prefit stage will not make it into the training stage.

In [11]:
prefit_filter = lambda error: abs(error) < 2

Under the hood, `DarkGreyBox` uses `lmfit.minimize` to fit the thermal model parameters to the training data by passing it a custom objective function defined by the model class. The fit method used by `lmfit.minimize` can be customised as well, and while the standard Nelder-Mead and Levenberg-Marquardt methods work well in general, it is worth experimenting with others (see here for a list of supported methods: https://lmfit.github.io/lmfit-py/fitting.html#fit-methods-table).

In [12]:
method = 'nelder'

Setting up is ready! We only need to call the `darkgreyfit` convenience pipeline function that delegates the work to the prefit, train and test low-level procedures running on multiple processes and reducing / cleaning the interim results. The returned `pandas.DataFrame` contains the fit models and their train/test performance evaluation.   

In [13]:
df = darkgreyfit(
    models,
    X_train,
    y_train,
    X_test,
    y_test,
    ic_params_map,
    error_metric=error_metric,
    prefit_splits=prefit_splits,
    prefit_filter=prefit_filter,
    reduce_train_results=True,
    method=method,
    n_jobs=-1,
    verbose=10
)

[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    4.7s remaining:    0.0s


In this specific example:
* *Initial Population:* 5 model structures were prefit to 28x 24-hour data chunks resulting in 140 pipelines
* *Fitness Function:* The RMSE of a prefit had to be < 2˚C to survive
* *Selection:* 110 pipelines survived the prefit selection and made it into training on the entire training set 
* (No *Crossover* or *Mutation*)
* *Termination:* Out of the 110 pipelines trained, 11 distinct model results were found (i.e. many pipelines were arriving at the same parameter set results)

In [ ]:
df

We can select the model with the lowest error and display its params

In [ ]:
df.loc[select_idx, (['train', 'test'], 'model_result')]

In [ ]:
select_idx = df[('train', 'error')].argmin()

model, error_train = df.loc[select_idx, ('train', ['model', 'error'])]
error_test = df.loc[select_idx, ('test', 'error')]
train_results, test_results = df.loc[select_idx, (['train', 'test'], 'model_result')]

print(f'\t\tTrain error: {error_train}')
model.result.params

The results show that the `TiTeThRia` model has a good fit on the training set (R2=0.9612) and the errors are low for both the training and test sets (RMSE 0.3719 and 0.3383 respectively) indicating that the model generalises well for data it hasn't seen before (no overfitting problem).

## Visualization

In [ ]:
# Plotting Train results
# ==============================================================================
fig = make_subplots(rows=1, cols=2, vertical_spacing=0.02, subplot_titles=(f'RMSE: {error_train}', ''))

fig.add_trace(go.Scatter(x=y_train.index, y=y_train, name='Ti Measured', mode='lines'), row=1, col=1)
for i, (key, value) in enumerate(train_results.var.items()):
    fig.add_trace(go.Scatter(x=y_train.index, y=value, name=f"{key} Modelled", mode='lines'), row=1, col=1)

# Plotting residuals
fig.add_trace(go.Scatter(x=y_train.index, y=y_train - train_results.Z, name='Residuals', mode='markers', marker=dict(color='black', opacity=0.5)), row=1, col=2)

fig.update_layout(title_text=f"Model: {model.__class__.__name__} - Train")

fig.show()

In [ ]:
# Plotting Test results
# ==============================================================================
fig = make_subplots(rows=1, cols=2, vertical_spacing=0.02, subplot_titles=(f'RMSE: {round(error_test, 2)}', ''))

fig.add_trace(go.Scatter(x=y_test.index, y=y_test, name='Ti Measured', mode='lines'), row=1, col=1)
for i, (key, value) in enumerate(test_results.var.items()):
    fig.add_trace(go.Scatter(x=y_test.index, y=value, name=f"{key} Modelled", mode='lines'), row=1, col=1)

# Plotting residuals
fig.add_trace(go.Scatter(x=y_test.index, y=y_test - test_results.Z, name='Residuals', mode='markers', marker=dict(color='black', opacity=0.5)), row=1, col=2)

fig.update_layout(title_text=f"Model: {model.__class__.__name__} - Test")

fig.show()

# References
We report here relevant references:
1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2